In [7]:
import pandas as pd
from config import app, db
from models import Property, Location, Address, Lot, Building, Sale, Class, User

SEED_DATA_ADDRESS = "/Users/mukkundsunjii/projects/proper-ty/seed/Enodo_Skills_Assessment_Data_File.xlsx"

properties_df = pd.read_excel(SEED_DATA_ADDRESS)

properties_df["SALE_DATE"] = properties_df["SALE_DATE"].apply(lambda x: pd.to_datetime(x, errors='coerce'))
properties_df["SALE_DATE"] = properties_df["SALE_DATE"].dt.date
properties_df["SALE_DATE"] = properties_df["SALE_DATE"].astype(object).where(properties_df["SALE_DATE"].notnull(), None)

users = [
    {
        "first_name": "Admin",
        "last_name": "User",
        "user_name": "admin",
    },
    {
        "first_name": "Basic",
        "last_name": "User",
        "user_name": "basic",
    }
]

ValueError: invalid literal for int() with base 10: '        '

In [14]:
building_class_dict = {}

for index, row in properties_df.iterrows():
    property_record = Property(estimated_market_value=row["ESTIMATED_MARKET_VALUE"], full_bath_count=row["FULL_BATH"], half_bath_count=row["HALF_BATH"], fireplace=row["FIREPLACE"], garage_description=row["GAR_DESC"])
    address = Address(tax_code=row["TAX_CODE"], city=row["CITY"], zip_code=row["Zip"], street=row["STREET"], house_number=row["HOUSENO"], neighborhood=row["NEIGHBORHOOD"], full_address=row["Full Address"])
    location = Location(latitude=row["Longitude"], longitude=row["Latitude"], address=address)
    property_record.location = location
    property_record.address = address
    lot = Lot(land_area=row["LAND_SQ_FT"], building_area=row["BUILDING_SQ_FT"])
    unit_count = row["UNITS_TOT"] if type(row["UNITS_TOT"]) is int else None
    commercial_unit_count = row["COMM_UNITS"] if type(row["COMM_UNITS"]) is int else None
    building = Building(building_type="RES_TYPE", usage=row["BLDG_USE"], apartment_count=row["APPCNT"], commercial_unit_count=row["COMM_UNITS"], external_description=row["EXT_DESC"], basement_description=row["BSMT_DESC"], attic_description=row["ATTIC_DESC"], ac_count=row["AC"], age=row["AGE"], unit_count=row["UNITS_TOT"], multi_sale=bool(row["MULTI_SALE"]), lot=lot)
    
    building_class = building_class_dict.get(row["OVACLS"])
    if building_class is None:
        building_class = Class(class_number=row["OVACLS"], description=row["CLASS_DESCRIPTION"])
        building_class_dict[row["OVACLS"]] = building_class
    building_class.buildings.append(building)
    
    property_record.building = building
    sale = Sale(sale_date=row["SALE_DATE"], sale_price=row["SALE_AMOUNT"], asset_property=property_record)

In [6]:
building_class_dict.keys()

dict_keys([211, 212, 315, 314, 318, 391, 225, 313, 991, 913, 996, 915, 396, 297])